In [1]:
import cv2

# === Load DNN model files ===
model_file = "res10_300x300_ssd_iter_140000_fp16.caffemodel"
config_file = "deploy.prototxt"
net = cv2.dnn.readNetFromCaffe(config_file, model_file)

# === Load video ===
cap = cv2.VideoCapture(r'C:\Users\veyadav\Downloads\Drupal-Branching.mp4' )

# === Target screen size ===
screen_width, screen_height = 1280, 720

cv2.namedWindow("DNN Face Detection", cv2.WINDOW_NORMAL)
cv2.setWindowProperty("DNN Face Detection", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Get frame size
    h, w = frame.shape[:2]

    # === Run face detection ===
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0,
                                 (300, 300), (104.0, 177.0, 123.0))
    net.setInput(blob)
    detections = net.forward()

    # === Parse results and draw boxes ===
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.7:
            box = detections[0, 0, i, 3:7] * [w, h, w, h]
            x1, y1, x2, y2 = box.astype("int")
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"{confidence:.2f}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    # === Resize frame to fit screen ===
    fh, fw = frame.shape[:2]
    scale = min(screen_width / fw, screen_height / fh)
    new_w, new_h = int(fw * scale), int(fh * scale)
    resized_frame = cv2.resize(frame, (new_w, new_h))

    # Center in a black canvas
    canvas = cv2.copyMakeBorder(
        resized_frame,
        top=(screen_height - new_h) // 2,
        bottom=(screen_height - new_h + 1) // 2,
        left=(screen_width - new_w) // 2,
        right=(screen_width - new_w + 1) // 2,
        borderType=cv2.BORDER_CONSTANT,
        value=[0, 0, 0]
    )

    cv2.imshow("DNN Face Detection", canvas)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()